# Quality Score Prediction with Stacking Ensemble

This notebook implements a stacking ensemble model to predict the `quality_score` for a manufacturing dataset. The model uses **CatBoost** and **LinearSVR** as base models, with **Linear Regression** as the meta-model. The code was used in a Kaggle competition, achieving a public leaderboard score of **0.10676**, but underperformed on the private leaderboard.

## Overview
- **Data**: Manufacturing data with features like `defect_area`, `plate_length`, `temperature`, etc.
- **Task**: Predict `quality_score` (continuous target variable).
- **Approach**:
  1. Feature engineering (e.g., logarithmic transformations, time-based features).
  2. Preprocessing (imputation, scaling).
  3. Hyperparameter tuning with Optuna.
  4. Stacking ensemble with CatBoost and LinearSVR.
  5. Submission file generation.

## Why Share?
Despite poor performance on the private leaderboard (due to overfitting and non-linear data mismatches), this code serves as a learning example for stacking ensembles, feature engineering, and hyperparameter optimization.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import optuna
import joblib
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

## Load the Data

We load the training and test datasets from CSV files. The `quality_score` is the target variable in the training set.

In [ ]:
# Load training and test data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Separate IDs and target variable
train_ids = train_df['id']  # Store training IDs
test_ids = test_df['id']    # Store test IDs
target = train_df['quality_score']  # Extract target variable

# Drop ID and target columns from training data, and ID from test data
train_df = train_df.drop(['id', 'quality_score'], axis=1)
test_df = test_df.drop(['id'], axis=1)

## Target Encoding

We apply target encoding to `operator_id` and `machine_id` by computing the mean `quality_score` for each category. This helps encode categorical variables numerically.

In [ ]:
# Compute mean quality_score for each operator and machine
operator_mean = train_df.groupby('operator_id')['quality_score'].mean()
machine_mean = train_df.groupby('machine_id')['quality_score'].mean()

# Save the encodings for future use
joblib.dump(operator_mean, 'operator_mean.pkl')
joblib.dump(machine_mean, 'machine_mean.pkl')

## Feature Engineering

We create new features to capture relationships in the data:
- Ratio-based features (e.g., `defect_per_length`).
- Logarithmic transformation of `defect_area`.
- Time-based features extracted from `production_date` and `production_time`.
- Target-encoded features for `operator_id` and `machine_id`.

In [ ]:
def advanced_feature_engineering(df, operator_mean, machine_mean):
    """Apply advanced feature engineering to the dataset.
    
    Args:
        df (pd.DataFrame): Input dataframe.
        operator_mean (pd.Series): Mean quality_score per operator.
        machine_mean (pd.Series): Mean quality_score per machine.
    
    Returns:
        pd.DataFrame: Transformed dataframe with new features.
    """
    df = df.copy()
    
    # Ratio-based features
    df['defect_per_length'] = df['defect_area'] / (df['plate_length'] + 1e-6)  # Avoid division by zero
    df['temp_times_thickness'] = df['temperature'] * df['plate_thickness']
    df['log_defect_area'] = np.log1p(df['defect_area'])  # Log transform defect_area
    df['thickness_squared'] = df['plate_thickness'] ** 2
    df['area_per_thickness'] = df['defect_area'] / (df['plate_thickness'] + 1e-6)
    df['temp_per_length'] = df['temperature'] / (df['plate_length'] + 1e-6)
    df['brightness_per_luminosity'] = df['brightness_index'] / (df['min_luminosity'] + 1e-6)
    df['edge_square_ratio'] = df['edge_index'] / (df['square_index'] + 1e-6)
    
    # Extract time-based features
    df['production_datetime'] = pd.to_datetime(df['production_date'] + ' ' + df['production_time'])
    df['day_of_week'] = df['production_datetime'].dt.dayofweek
    df['month'] = df['production_datetime'].dt.month
    df['hour'] = df['production_datetime'].dt.hour
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['shift'] = pd.cut(df['hour'], bins=[0, 8, 16, 24], labels=[1, 2, 3], include_lowest=True)
    
    # Drop original datetime columns
    df = df.drop(['production_date', 'production_time', 'production_datetime'], axis=1)
    
    # Apply target encoding
    df['operator_encoded'] = df['operator_id'].map(operator_mean).fillna(operator_mean.mean())
    df['machine_encoded'] = df['machine_id'].map(machine_mean).fillna(machine_mean.mean())
    
    # Drop original categorical columns
    df = df.drop(['operator_id', 'machine_id'], axis=1)
    return df

# Apply feature engineering to both train and test sets
train_df = advanced_feature_engineering(train_df, operator_mean, machine_mean)
test_df = advanced_feature_engineering(test_df, operator_mean, machine_mean)

## Preprocessing

We preprocess the data by:
- Imputing missing values using KNN Imputer.
- Scaling features with StandardScaler.

In [ ]:
# Impute missing values using KNN
imputer = KNNImputer(n_neighbors=5)
train_df = pd.DataFrame(imputer.fit_transform(train_df), columns=train_df.columns)
test_df = pd.DataFrame(imputer.transform(test_df), columns=test_df.columns)
joblib.dump(imputer, 'imputer.pkl')  # Save the imputer

# Scale features
scaler = StandardScaler()
train_df_scaled = scaler.fit_transform(train_df)
test_df_scaled = scaler.transform(test_df)
joblib.dump(scaler, 'scaler.pkl')  # Save the scaler

## Hyperparameter Tuning with Optuna

We use Optuna to tune hyperparameters for both base models:
- **CatBoost**: A gradient boosting model.
- **LinearSVR**: A linear Support Vector Regressor.

In [ ]:
# Define objective function for CatBoost
def objective_catboost(trial):
    """Objective function for tuning CatBoost hyperparameters."""
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000, step=10),
        'depth': trial.suggest_int('depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, step=0.5),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1, step=0.1),
        'border_count': trial.suggest_int('border_count', 32, 255, step=5)
    }
    model = CatBoostRegressor(**param, random_state=42, verbose=0)
    scores = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in kf.split(train_df_scaled):
        X_train, X_val = train_df_scaled[train_idx], train_df_scaled[val_idx]
        y_train, y_val = target.iloc[train_idx], target.iloc[val_idx]
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)
        preds = model.predict(X_val)
        scores.append(np.sqrt(mean_squared_error(y_val, preds)))
    return np.mean(scores)

# Define objective function for LinearSVR
def objective_linearsvr(trial):
    """Objective function for tuning LinearSVR hyperparameters."""
    param = {
        'C': trial.suggest_float('C', 0.01, 10.0, log=True),
        'epsilon': trial.suggest_float('epsilon', 0.01, 0.5),
        'max_iter': 10000
    }
    model = LinearSVR(**param, random_state=42)
    scores = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in kf.split(train_df_scaled):
        X_train, X_val = train_df_scaled[train_idx], train_df_scaled[val_idx]
        y_train, y_val = target.iloc[train_idx], target.iloc[val_idx]
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        scores.append(np.sqrt(mean_squared_error(y_val, preds)))
    return np.mean(scores)

# Tune CatBoost
study_catboost = optuna.create_study(direction='minimize')
study_catboost.optimize(objective_catboost, n_trials=50)
best_catboost = CatBoostRegressor(**study_catboost.best_params, random_state=42, verbose=0)
joblib.dump(best_catboost, 'catboost_model.pkl')
print(f"Best CatBoost params: {study_catboost.best_params}")

# Tune LinearSVR
study_linearsvr = optuna.create_study(direction='minimize')
study_linearsvr.optimize(objective_linearsvr, n_trials=50)
best_linearsvr = LinearSVR(**study_linearsvr.best_params, random_state=42, max_iter=10000)
joblib.dump(best_linearsvr, 'linearsvr_model.pkl')
print(f"Best LinearSVR params: {study_linearsvr.best_params}")

## Stacking Ensemble

We use a stacking ensemble:
- **Base Models**: CatBoost and LinearSVR.
- **Meta-Model**: Linear Regression.
- **Cross-Validation**: 5-fold KFold for out-of-fold predictions.

In [ ]:
# Define base models
base_models = [
    ('CatBoost', best_catboost),
    ('LinearSVR', best_linearsvr)
]

# Initialize arrays for stacking
kf = KFold(n_splits=5, shuffle=True, random_state=42)
stacking_train = np.zeros((len(train_df_scaled), len(base_models)))
stacking_test = np.zeros((len(test_df_scaled), len(base_models)))

# Generate out-of-fold predictions for stacking
for i, (name, model) in enumerate(base_models):
    print(f"Training {name}...")
    oof_preds = np.zeros(len(train_df_scaled))
    test_preds = np.zeros(len(test_df_scaled))
    for train_idx, val_idx in kf.split(train_df_scaled):
        X_train, X_val = train_df_scaled[train_idx], train_df_scaled[val_idx]
        y_train, y_val = target.iloc[train_idx], target.iloc[val_idx]
        if name == 'CatBoost':
            model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)
        else:
            model.fit(X_train, y_train)
        oof_preds[val_idx] = model.predict(X_val)
        test_preds += model.predict(test_df_scaled) / kf.n_splits
    stacking_train[:, i] = oof_preds
    stacking_test[:, i] = test_preds
    rmse = np.sqrt(mean_squared_error(target, oof_preds))
    print(f"{name} Validation RMSE: {rmse:.5f}")

# Train meta-model
meta_model = LinearRegression()
meta_model.fit(stacking_train, target)
joblib.dump(meta_model, 'meta_model.pkl')

# Generate final predictions
final_preds = meta_model.predict(stacking_test)

## Generate Submission File

We create a submission file with the predicted `quality_score` for the test set.

In [ ]:
# Create submission dataframe
submission_df = pd.DataFrame({'id': test_ids, 'quality_score': final_preds})
submission_df.to_csv('catboost_linearsvr_submission.csv', index=False)
print("Submission file 'catboost_linearsvr_submission.csv' created successfully!")

## Prediction Function for New Data

This function allows predicting `quality_score` for new data using the trained stacking ensemble.

In [ ]:
def predict_new_data(new_data, base_models, meta_model_path='meta_model.pkl', 
                     imputer_path='imputer.pkl', scaler_path='scaler.pkl', 
                     operator_mean_path='operator_mean.pkl', machine_mean_path='machine_mean.pkl'):
    """Predict quality_score for new data using the Stacking Ensemble.
    
    Args:
        new_data (pd.DataFrame): New data to predict.
        base_models (list): List of trained base models.
        meta_model_path (str): Path to meta-model file.
        imputer_path (str): Path to Imputer file.
        scaler_path (str): Path to Scaler file.
        operator_mean_path (str): Path to Target Encoding for operator_id.
        machine_mean_path (str): Path to Target Encoding for machine_id.
    
    Returns:
        pd.DataFrame or np.array: Predicted quality_scores.
    """
    # Load saved preprocessing objects
    meta_model = joblib.load(meta_model_path)
    imputer = joblib.load(imputer_path)
    scaler = joblib.load(scaler_path)
    operator_mean = joblib.load(operator_mean_path)
    machine_mean = joblib.load(machine_mean_path)

    # Extract IDs if present
    new_ids = new_data['id'] if 'id' in new_data else None
    
    # Apply feature engineering
    new_data = advanced_feature_engineering(new_data, operator_mean, machine_mean)
    
    # Preprocess new data
    new_data = pd.DataFrame(imputer.transform(new_data), columns=new_data.columns)
    new_data_scaled = scaler.transform(new_data)
    
    # Generate predictions from base models
    stacking_new = np.zeros((len(new_data_scaled), len(base_models)))
    for i, (name, model) in enumerate(base_models):
        stacking_new[:, i] = model.predict(new_data_scaled)
    
    # Predict with meta-model
    predictions = meta_model.predict(stacking_new)
    
    # Return predictions with IDs if provided
    if new_ids is not None:
        return pd.DataFrame({'id': new_ids, 'quality_score': predictions})
    return predictions

## Lessons Learned

This model achieved a decent score on the public leaderboard (0.10676) but performed poorly on the private leaderboard due to:
- **Overfitting to public data**: Features like `log_defect_area` and time-based features caused overfitting.
- **LinearSVR limitations**: The dataset was non-linear, and LinearSVR was not the best choice.
- **Lack of model diversity**: Adding non-linear models like LightGBM or SVR with RBF kernel could have improved performance.

For future competitions, consider:
- Analyzing data distribution to choose appropriate models.
- Using more diverse models in the ensemble.
- Avoiding features that may overfit to the public test set.